# YOLO 11 Model Training with Data Augmentation

This notebook trains a YOLO 11 model on the animal detection dataset with data augmentation.


In [ ]:
# Import necessary libraries
from ultralytics import YOLO
import os
from pathlib import Path
import yaml
import torch

# Set up paths
dataset_path = Path("d:/labbd-2/Downloads/TesisAnimales.v3i.yolov12")
data_yaml = dataset_path / "data.yaml"

# Create organized folder structure for outputs
output_base = Path("models/yolo11")
output_base.mkdir(parents=True, exist_ok=True)

# Check device availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Libraries imported successfully!")
print(f"Device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Dataset path: {dataset_path.absolute()}")
print(f"Data YAML: {data_yaml.absolute()}")
print(f"Output directory: {output_base.absolute()}")

print(f"¿Torch detecta CUDA?: {torch.cuda.is_available()}")
print(f"Versión de CUDA en Torch: {torch.version.cuda}")


## Step 1: Verify Dataset Configuration


In [ ]:
# Verify data.yaml configuration
with open(data_yaml, 'r') as f:
    config = yaml.safe_load(f)
    
print("Dataset Configuration:")
print(f"Number of classes: {config['nc']}")
print(f"Class names: {config['names']}")
print(f"Train path: {config['train']}")
print(f"Validation path: {config['val']}")
print(f"Test path: {config['test']}")

# Verify paths exist
train_path = dataset_path / config['train']
val_path = dataset_path / config['val']
test_path = dataset_path / config['test']

print(f"\nPath verification:")
print(f"Train images exist: {train_path.exists()}")
print(f"Validation images exist: {val_path.exists()}")
print(f"Test images exist: {test_path.exists()}")


## Step 2: Initialize YOLO 11 Model


In [ ]:
# Initialize YOLO 11 model
# Using yolov11n (nano) for faster training, can be changed to yolov11s, yolov11m, yolov11l, yolov11x
model = YOLO('yolo11n.pt')  # Downloads pretrained weights automatically

print("YOLO 11 model initialized!")
print(f"Model type: {model.task}")


## Step 3: Configure Data Augmentation

Data augmentation helps improve model generalization by creating variations of training images.


In [ ]:
# Configure data augmentation parameters
# These augmentations will be applied during training
augmentation_config = {
    'hsv_h': 0.015,      # Hue augmentation (0-1)
    'hsv_s': 0.7,        # Saturation augmentation (0-1)
    'hsv_v': 0.4,        # Value augmentation (0-1)
    'degrees': 10.0,     # Rotation degrees
    'translate': 0.1,    # Translation (fraction of image size)
    'scale': 0.5,        # Scale augmentation (0-1)
    'shear': 2.0,        # Shear degrees
    'perspective': 0.0,  # Perspective augmentation (0-0.001)
    'flipud': 0.0,       # Vertical flip probability (0-1)
    'fliplr': 0.5,       # Horizontal flip probability (0-1)
    'mosaic': 1.0,       # Mosaic augmentation probability (0-1)
    'mixup': 0.1,        # Mixup augmentation probability (0-1)
    'copy_paste': 0.1,   # Copy-paste augmentation probability (0-1)
}

print("Data Augmentation Configuration:")
for key, value in augmentation_config.items():
    print(f"  {key}: {value}")


## Step 4: Train the Model


In [ ]:
# Train the model with data augmentation
# Outputs will be saved in models/yolo11/training directory
results = model.train(
    data=str(data_yaml.absolute()),  # Path to data.yaml
    epochs=100,                       # Number of training epochs
    imgsz=640,                        # Image size
    batch=16,                         # Batch size (adjust based on GPU memory)
    device=device,                    # Use GPU if available (set in first cell)
    workers=8,                        # Number of worker threads
    project=str(output_base),         # Project directory (models/yolo11)
    name='training',                  # Experiment name
    exist_ok=True,                    # Overwrite existing experiment
    pretrained=True,                  # Use pretrained weights
    optimizer='AdamW',                # Optimizer
    verbose=True,                     # Verbose output
    seed=42,                          # Random seed for reproducibility
    deterministic=True,               # Deterministic training
    single_cls=False,                 # Single class mode
    rect=False,                       # Rectangular training
    cos_lr=False,                     # Cosine LR scheduler
    close_mosaic=10,                  # Disable mosaic augmentation in last N epochs
    resume=False,                     # Resume from last checkpoint
    amp=True,                         # Automatic Mixed Precision
    fraction=1.0,                     # Dataset fraction to use
    profile=False,                    # Profile ONNX and TensorRT speeds
    freeze=None,                      # Freeze layers
    # Augmentation parameters
    **augmentation_config
)

print("\nTraining completed!")
print(f"Best model saved at: {results.save_dir}")
print(f"Best weights: {results.save_dir / 'weights' / 'best.pt'}")
print(f"Last weights: {results.save_dir / 'weights' / 'last.pt'}")


In [ ]:
# Evaluate the model on validation set
metrics = model.val()
print("\nValidation Metrics:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")


## Step 6: Export the Model

Export the trained model to various formats (ONNX, TensorRT, CoreML, etc.)


In [ ]:
# Export model to different formats
# Create exports directory within the organized folder structure
export_dir = output_base / "exports"
export_dir.mkdir(exist_ok=True)

# Check platform
import platform
is_windows = platform.system() == 'Windows'
is_macos = platform.system() == 'Darwin'

# Export to ONNX (recommended for deployment)
print("Exporting to ONNX format...")
onnx_path = model.export(format='onnx', imgsz=640, optimize=True)
# Move to organized exports folder
import shutil
onnx_dest = export_dir / "yolo11_animal_detection.onnx"
if Path(onnx_path).exists():
    shutil.move(onnx_path, onnx_dest)
    onnx_path = onnx_dest
print(f"ONNX model saved at: {onnx_path}")

# Export to TensorRT (for NVIDIA GPUs)
# Uncomment if you have TensorRT installed
# print("Exporting to TensorRT format...")
# trt_path = model.export(format='engine', imgsz=640)
# trt_dest = export_dir / "yolo11_animal_detection.engine"
# if Path(trt_path).exists():
#     shutil.move(trt_path, trt_dest)
#     trt_path = trt_dest
# print(f"TensorRT model saved at: {trt_path}")

# Export to CoreML (for Apple devices - macOS/Linux only)
if not is_windows:
    print("Exporting to CoreML format...")
    try:
        coreml_path = model.export(format='coreml', imgsz=640)
        coreml_dest = export_dir / "yolo11_animal_detection.mlpackage"
        if Path(coreml_path).exists():
            shutil.move(coreml_path, coreml_dest)
            coreml_path = coreml_dest
        print(f"CoreML model saved at: {coreml_path}")
    except Exception as e:
        print(f"CoreML export failed: {e}")
else:
    print("Skipping CoreML export (not supported on Windows - requires macOS or Linux)")

# Export to TensorFlow Lite (for mobile devices)
# Note: TensorFlow Lite export may have compatibility issues on Windows
# ONNX format (exported above) is recommended for cross-platform deployment
print("Exporting to TensorFlow Lite format...")
try:
    tflite_path = model.export(format='tflite', imgsz=640)
    tflite_dest = export_dir / "yolo11_animal_detection.tflite"
    if Path(tflite_path).exists():
        shutil.move(tflite_path, tflite_dest)
        tflite_path = tflite_dest
    print(f"TensorFlow Lite model saved at: {tflite_path}")
except Exception as e:
    print(f"TensorFlow Lite export failed: {e}")
    print("Note: TensorFlow Lite export has known compatibility issues on Windows.")
    print("The ONNX format (exported above) is recommended for deployment and works on all platforms.")

print("\nAll exports completed!")
print(f"\nSummary:")
print(f"✓ ONNX model: {onnx_path}")
if not is_windows and 'coreml_path' in locals():
    print(f"✓ CoreML model: {coreml_path}")
if 'tflite_path' in locals() and Path(tflite_path).exists():
    print(f"✓ TensorFlow Lite model: {tflite_path}")
print(f"\nAll exports saved in: {export_dir.absolute()}")
print("Note: ONNX is the recommended format for cross-platform deployment.")


## Step 7: Test the Exported Model (Optional)

Test the exported ONNX model to verify it works correctly.


In [ ]:
# Load and test the exported ONNX model
from ultralytics import YOLO

# Load the exported ONNX model
onnx_model = YOLO(str(onnx_path))

# Create test results directory
test_results_dir = output_base / "test_results"
test_results_dir.mkdir(exist_ok=True)

# Test on a sample image
test_image_path = dataset_path / "test" / "images"
if test_image_path.exists():
    test_images = list(test_image_path.glob("*.jpg"))[:3]  # Test on first 3 images
    if test_images:
        print(f"Testing on {len(test_images)} sample images...")
        results = onnx_model.predict(
            source=[str(img) for img in test_images],
            save=True,
            save_txt=True,
            conf=0.25,
            iou=0.45,
            project=str(output_base),
            name="test_results"
        )
        print(f"Predictions completed! Check {test_results_dir.absolute()} for results.")
    else:
        print("No test images found.")
else:
    print("Test images directory not found.")
